In [106]:
import pandas as pd
import numpy as np
import math
import json

In [107]:
data_home = "./"

In [108]:
dict_user_u = {}
for line in open(data_home+"new_triplet_dataset_sum_df_test_1.csv", "r", encoding="utf-8"):
    #根据head（）来改动
    user_id,goods_id,click_count_goods, add_cart_count, order_count, click_count_sum, score= line.strip().split(',')
    #itemSim.setdefault(goods_id, 0)
    dict_user_u.setdefault(str(user_id), {})    
    dict_user_u[user_id][goods_id] = score

In [109]:
del dict_user_u['user_id']

In [1]:
#dict_user_u

In [110]:
class UserCF:
    def __init__(self, dict_user_u):
        self.user_score_dict = dict_user_u
        #self.users_sim = self.userSimilarity()
        #self.users_sim = self.userSimilarityBetter()
        self.users_sim = self.UserSimilarityBest()
        
    # 计算用户之间的相似度,采用的是遍历每一个用户进行计算
    def userSimilarity(self):
        W = dict()
        for u in self.user_score_dict.keys():
            W.setdefault(u,{})
            for v in self.user_score_dict.keys():
                if u == v:
                    continue
                u_set = set([key for key in self.user_score_dict[u].keys() if float(self.user_score_dict[u][key]) > 0])
                v_set = set([key for key in self.user_score_dict[v].keys() if float(self.user_score_dict[v][key]) > 0])
                num_1 = float(len(u_set & v_set))
                W[u][v] = num_1 / (len(u_set) + len(v_set))
        return W

    # 计算用户之间的相似度，采用优化算法时间复杂度的方法
    def userSimilarityBetter(self):
        # 得到每个item被哪些user评价过
        item_users = dict()
        for u, items in self.user_score_dict.items():
            for i in items.keys():
                item_users.setdefault(i,set())
                if float(self.user_score_dict[u][i]) > 0:
                    item_users[i].add(u)
        # 构建倒排表
        C = dict()
        N = dict()
        for i, users in item_users.items():
            for u in users:
                N.setdefault(u,0)
                N[u] += 1
                C.setdefault(u,{})
                for v in users:
                    C[u].setdefault(v, 0)
                    if u == v:
                        continue
                    C[u][v] += 1
        
        # 构建相似度矩阵
        W = dict()
        for u, related_users in C.items():
            W.setdefault(u,{})
            for v, cuv in related_users.items():
                if u==v:
                    continue
                W[u].setdefault(v, 0.0)
                W[u][v] = cuv / (N[u] * N[v])
        return W

    # 计算用户之间的相似度，采用惩罚热门商品和优化算法复杂度的算法
    def UserSimilarityBest(self):
        # 得到每个item被哪些user评价过
        item_users = dict()
        for u, items in self.user_score_dict.items():
            for i in items.keys():
                item_users.setdefault(i,set())
                #if float(self.user_score_dict[u][i]) > 0:
                item_users[i].add(u)
        # 构建倒排表
        C = dict()
        N = dict()
        for i, users in item_users.items():
            for u in users:
                N.setdefault(u,0)
                N[u] += 1
                C.setdefault(u,{})
                for v in users:
                    C[u].setdefault(v, 0)
                    if u == v:
                        continue
                    C[u][v] += 1 / math.log(1+len(users))
        # 构建相似度矩阵
        W = dict()
        for u, related_users in C.items():
            W.setdefault(u,{})
            for v, cuv in related_users.items():
                if u==v:
                    continue
                W[u].setdefault(v, 0.0)
                W[u][v] = cuv / math.sqrt(N[u] * N[v])
        return W
    # 预测用户对item的评分
    def preUserItemScore(self, userA, item):
        score = 0.0
        for user in self.users_sim[userA].keys():
            if user != userA:
                try:
                    num = float(self.user_score_dict[user][item])
                except:
                    num = 0                    
                score += (float(self.users_sim[userA][user]) * num)
        return score

    # 为用户推荐物品
    def recommend(self, userA):
        # 计算userA 未评分item的可能评分
        user_item_score_dict = dict()
        for item in self.user_score_dict[userA].keys():
            if float(self.user_score_dict[userA][item]) <= 0:
                user_item_score_dict[item] = self.preUserItemScore(userA, item)
        return user_item_score_dict

    

if __name__ == "__main__":
    ub = UserCF(dict_user_u)
    #print(ub.users_sim)
    #print(ub.recommend('33689928'))

    


KeyboardInterrupt: 

In [102]:
dict_predict_result_u = {}

In [103]:
for i in dict_user_u.keys(): 
    dict_predict_result_u[i]=sorted(ub.recommend(i).items(), key=lambda v: v[1])[::-1][:4]
        

In [104]:
len(dict_predict_result_u.keys())

22195

In [ ]:
file_u=open('dict_predict_price_result_u_1.json',mode='w')
file_u.writelines(json.dumps(dict_predict_result_u)+',\n' +'\n')
file_u.close()